# Welcome to this TC-Python tutorial

Please click the run button in the toolbar or click shift+enter hot-key to run the codes in a cell.

## Init a session
Note: *The cell below should be  invoked only once in the lifetime of the process (kernel).*
To end the session before closing the kernel, remove all python variables of `session`. Such as 

    del session
    # or 
    session = []
   

In [1]:
import tc_ipython as tc
session = tc.TCPython_session()

In [2]:
# You should see a Java process running in the backgroud. That's the Thermo-Calc server
! ps -U $USER

    PID TTY          TIME CMD
  46102 ?        00:00:00 condor_exec.exe
  46121 ?        00:00:00 bash
  46376 ?        00:00:26 ZMQbg/1
  49668 ?        00:00:00 ZMQbg/1
  49709 ?        00:00:00 java
  49763 pts/6    00:00:00 ps


## Define system
Each cell below is preceded by the eqivalent console mode commands.

    sw tcfe9
    d-sys fe c mn

In [3]:
sysconf=session.select_database_and_elements("TCFE9", ['Fe', 'C', 'Mn'])

    rej ph *
    res phase  BCC_A2 FCC_A1 GRAPHITE KSI_CARBIDE CEMENTITE

In [4]:
sysconf.without_default_phases()
sysconf.select_phase('BCC_A2 FCC_A1 GRAPHITE KSI_CARBIDE CEMENTITE' );

    get-data
    list-system phases

In [6]:
fesys=sysconf.get_system()
fesys.get_phases_in_system()

['CEMENTITE', 'BCC_A2', 'FCC_A1', 'GRAPHITE', 'KSI_CARBIDE']

## Set conditions for poly3 calculations

    go poly
    set-condition ...
    ADVANCED_OPTIONS GLOBAL_MINIMIZATION N
    

In [7]:
conditions={
    "P"      : 101325,
    "N"      : 1.0,
    "T"      : 773,
    "W(C)"   : 0.22E-2,
    "W(Mn)"  : 1.25E-2
}
calc_eq = fesys.with_single_equilibrium_calculation()
for k,v in conditions.items():
    calc_eq.set_condition(k,v)
calc_eq = calc_eq.disable_global_minimization()

This will print out the conditions to the log file under `~/.Thermo-Calc`

In [15]:
calc_eq.run_poly_command('LIST_CONDITIONS');
!tail -n 3 "$HOME/.Thermo-Calc/thermo-calc.log"

20:47:11,733 [Thread-2] INFO  CalculationContainer:   DEGREES OF FREEDOM 0
20:48:17,788 [Thread-2] INFO  CalculationContainer:   P=1.01325E5, N=1, T=773, W(C)=2.2E-3, W(MN)=1.25E-2
20:48:17,788 [Thread-2] INFO  CalculationContainer:   DEGREES OF FREEDOM 0


## Run the calculation

In [16]:
r=calc_eq.calculate()
r.get_stable_phases()

['BCC_A2#1', 'CEMENTITE#1', 'GRAPHITE#1']

## List information of each stable phases

In [17]:
from IPython.core.display import display, HTML
cols = [
    ('Mole','NP({ph})'),
    ('C mass fr.', 'W({ph},C)'), 
    ('Mn mass fr.', 'W({ph},Mn)'), 
    ('C chem. potential', 'MUR(C,{ph})'), 
    ('Mn chem. potential', 'MUR(Mn,{ph})'),
    ('Molar Gibbs energy', 'GM({ph})')
]
df=tc.to_phase_table(r,cols)
display(HTML(df.to_html(float_format=lambda x:'{0:12.3e}'.format(x))))

,Mole,C mass fr.,Mn mass fr.,C chem. potential,Mn chem. potential,Molar Gibbs energy
BCC_A2#1,9.751e-01,3.832e-06,7.899e-03,nan,-2.254e+04,-2.864e+04
CEMENTITE#1,1.971e-02,6.722e-02,2.980e-01,nan,nan,-2.925e+04
GRAPHITE#1,5.202e-03,1.000e+00,0.000e+00,0.000e+00,nan,-7.696e+03


## Perform some math ...

In [22]:
print( 'sum(N*GM)='  ,  (df['Mole']*df['Molar Gibbs energy']).sum() , ' = G = ' , r.get_value_of('G') )

sum(N*GM)= -28541.734449276668  = G =  -28541.734449276668


## Destroy the TC session (will close the ThermoCalc server in the background)

In [23]:
session=0

In [25]:
# This will cause error:
calc_eq.calculate()

APIServerException: Connection to the internal API-server failed